In [ ]:
# reproducable randomness
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
from neural_stylization.reconstruct_content import reconstruct_content
from neural_stylization.optimizers import SGD, L_BFGS
from neural_stylization.util.build_callback import build_callback

# SGD

In [ ]:
from keras import backend as K

In [ ]:
block4_conv2_sgd = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    optimize=SGD(learning_rate=1e-3),
    iterations=100,
    callback=build_callback('build/content/block4_conv2/sgd')
)

# L-BFGS

In [ ]:
block4_conv2_lbfgs = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/content/block4_conv2/lbfgs')
)
block4_conv2_lbfgs.save('tex/img/content/block4_conv2.png')

## Visualization of all 5 Gatys et al. Content Layers

In [ ]:
block1_conv1 = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    layer_name='block1_conv1',
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/content/block1_conv1')
)
block1_conv1.save('tex/img/content/block1_conv1.png')

In [ ]:
block2_conv1 = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    layer_name='block2_conv1',
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/content/block2_conv1')
)
block2_conv1.save('tex/img/content/block2_conv1.png')

In [ ]:
block3_conv1 = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    layer_name='block3_conv1',
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/content/block3_conv1')
)
block3_conv1.save('tex/img/content/block3_conv1.png')

In [ ]:
block4_conv1 = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    layer_name='block4_conv1',
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/content/block4_conv1')
)
block4_conv1.save('tex/img/content/block4_conv1.png')

In [ ]:
block5_conv1 = reconstruct_content(
    content_path='img/content/tubingen.jpg',
    image_shape=(500, 300),
    layer_name='block5_conv1',
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/content/block5_conv1')
)
block5_conv1.save('tex/img/content/block5_conv1.png')